## Data Analysis Trade IronFX ('Broker')

## Objetivo

Encontrar un patron en los datos tal que pueda predecir que tipo de operaciones en divisas son más probables a ser ganadoras y entender cuál fue el problemas en los trades perdedores.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
dft0=pd.read_csv('diario_trading_real.csv', decimal=',')
dft0

,Time_open,Type,Size,Symbol,Price_open,S/L,T/P,Time_close,Price_close,Swap,Profit,Comment,Unnamed: 12
0,2021/03/31 12:00:14,sell,0.01,gbpusd_M15,1.37986,1.38092,1.37825,2021/03/31 12:11:11,1.38092,0,-1.06,Me senti relajado,NaN
1,2021/03/31 12:31:33,buy,0.03,usdjpy_M15,110.657,110.53,110.71,2021/03/31 13:10:36,110.71,0,1.44,senti control,NaN
2,2021/03/31 12:33:46,sell,0.03,gpbusd_M5,1.37972,1.38072,1.37864,2021/03/31 13:24:21,1.37864,0,3.24,sentí control,NaN
3,2021/03/31 16:11:22,sell,0.03,usdjpy_M15,110.663,110.822,110.56,2021/03/31 18:08:40,110.822,0,-4.3,Senti relajado,NaN
4,2021/03/31 19:56:49,buy,0.03,usdchf_M30,0.94478,0.94357,0.94533,2021/03/31 11:23:01,0.94533,0,1.75,Senti control,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,2022/04/25 12:19:32,buy,0.01,uk100_m2_H1,7382,7322,7442,2022/04/26 1:00:03,7442,-0.03,7.65,Cabeza fría,NaN
331,2022/04/28 17:44:31,sell,0.01,usdjpy_H1,130.825,131.342,130.237,2022/04/29 1:23:21,130.237,0,4.51,Cabeza fria,NaN
332,2022/05/05 10:02:04,sell,0.01,wticash_H1,107.24,109.46,105.02,2022/05/05 16:28:07,109.46,-0.55,-22.2,Cabeza fría,NaN
333,2022/05/09 14:01:15,buy,0.01,ger40_m2_H1,13475,13336,13622,2022/05/10 3:38:38,13622,-0.05,38.83,Cabeza fría,NaN


In [3]:
dft0.drop('Unnamed: 12', inplace=True, axis=1)

In [4]:
dft0

,Time_open,Type,Size,Symbol,Price_open,S/L,T/P,Time_close,Price_close,Swap,Profit,Comment
0,2021/03/31 12:00:14,sell,0.01,gbpusd_M15,1.37986,1.38092,1.37825,2021/03/31 12:11:11,1.38092,0,-1.06,Me senti relajado
1,2021/03/31 12:31:33,buy,0.03,usdjpy_M15,110.657,110.53,110.71,2021/03/31 13:10:36,110.71,0,1.44,senti control
2,2021/03/31 12:33:46,sell,0.03,gpbusd_M5,1.37972,1.38072,1.37864,2021/03/31 13:24:21,1.37864,0,3.24,sentí control
3,2021/03/31 16:11:22,sell,0.03,usdjpy_M15,110.663,110.822,110.56,2021/03/31 18:08:40,110.822,0,-4.3,Senti relajado
4,2021/03/31 19:56:49,buy,0.03,usdchf_M30,0.94478,0.94357,0.94533,2021/03/31 11:23:01,0.94533,0,1.75,Senti control
...,...,...,...,...,...,...,...,...,...,...,...,...
330,2022/04/25 12:19:32,buy,0.01,uk100_m2_H1,7382,7322,7442,2022/04/26 1:00:03,7442,-0.03,7.65,Cabeza fría
331,2022/04/28 17:44:31,sell,0.01,usdjpy_H1,130.825,131.342,130.237,2022/04/29 1:23:21,130.237,0,4.51,Cabeza fria
332,2022/05/05 10:02:04,sell,0.01,wticash_H1,107.24,109.46,105.02,2022/05/05 16:28:07,109.46,-0.55,-22.2,Cabeza fría
333,2022/05/09 14:01:15,buy,0.01,ger40_m2_H1,13475,13336,13622,2022/05/10 3:38:38,13622,-0.05,38.83,Cabeza fría


In [5]:
dft0['Type'].unique()

array(['sell', 'buy', 'sell ', 'sel', 'buy '], dtype=object)

In [6]:
type1={'sell':'sell', 'buy':'buy', 'sel':'sell', 'buy ':'buy'}

In [7]:
dft0['Type']=dft0['Type'].map(type1)
dft0.groupby('Type').size()

Type
buy     164
sell    167
dtype: int64

Hay que tener en cuenta que durante un año de trading se realizarón más ordenes de venta que de compra

In [8]:
dft0['Symbol']

0       gbpusd_M15
1       usdjpy_M15
2        gpbusd_M5
3       usdjpy_M15
4       usdchf_M30
          ...     
330    uk100_m2_H1
331      usdjpy_H1
332     wticash_H1
333    ger40_m2_H1
334      nzdusd_H1
Name: Symbol, Length: 335, dtype: object

podemos ver en la columna de "Symbol" que se encuentra el par de divisas que se realizo la operación junto con un distintivo M (minutos) y H (horas). Procederemos a realizar esa separación para tener más precisión en el análisis

In [9]:
dft0[['Symbol_only','temporality0','temporality1','temporality2']]=dft0['Symbol'].str.split('_', expand=True)
dft0

,Time_open,Type,Size,Symbol,Price_open,S/L,T/P,Time_close,Price_close,Swap,Profit,Comment,Symbol_only,temporality0,temporality1,temporality2
0,2021/03/31 12:00:14,sell,0.01,gbpusd_M15,1.37986,1.38092,1.37825,2021/03/31 12:11:11,1.38092,0,-1.06,Me senti relajado,gbpusd,M15,None,None
1,2021/03/31 12:31:33,buy,0.03,usdjpy_M15,110.657,110.53,110.71,2021/03/31 13:10:36,110.71,0,1.44,senti control,usdjpy,M15,None,None
2,2021/03/31 12:33:46,sell,0.03,gpbusd_M5,1.37972,1.38072,1.37864,2021/03/31 13:24:21,1.37864,0,3.24,sentí control,gpbusd,M5,None,None
3,2021/03/31 16:11:22,sell,0.03,usdjpy_M15,110.663,110.822,110.56,2021/03/31 18:08:40,110.822,0,-4.3,Senti relajado,usdjpy,M15,None,None
4,2021/03/31 19:56:49,buy,0.03,usdchf_M30,0.94478,0.94357,0.94533,2021/03/31 11:23:01,0.94533,0,1.75,Senti control,usdchf,M30,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,2022/04/25 12:19:32,buy,0.01,uk100_m2_H1,7382,7322,7442,2022/04/26 1:00:03,7442,-0.03,7.65,Cabeza fría,uk100,m2,H1,None
331,2022/04/28 17:44:31,sell,0.01,usdjpy_H1,130.825,131.342,130.237,2022/04/29 1:23:21,130.237,0,4.51,Cabeza fria,usdjpy,H1,None,None
332,2022/05/05 10:02:04,sell,0.01,wticash_H1,107.24,109.46,105.02,2022/05/05 16:28:07,109.46,-0.55,-22.2,Cabeza fría,wticash,H1,None,None
333,2022/05/09 14:01:15,buy,0.01,ger40_m2_H1,13475,13336,13622,2022/05/10 3:38:38,13622,-0.05,38.83,Cabeza fría,ger40,m2,H1,None


In [10]:
dft0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335 entries, 0 to 334
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Time_open     335 non-null    object
 1   Type          331 non-null    object
 2   Size          335 non-null    object
 3   Symbol        335 non-null    object
 4   Price_open    335 non-null    object
 5   S/L           335 non-null    object
 6   T/P           335 non-null    object
 7   Time_close    335 non-null    object
 8   Price_close   335 non-null    object
 9   Swap          335 non-null    object
 10  Profit        335 non-null    object
 11  Comment       335 non-null    object
 12  Symbol_only   335 non-null    object
 13  temporality0  334 non-null    object
 14  temporality1  88 non-null     object
 15  temporality2  22 non-null     object
dtypes: object(16)
memory usage: 42.0+ KB


En la lista anterior observamos que los registros que contienen tipo de dato numerico se encuntran categorizados como tipo objeto, realizamos una transformación a este tipo de datos para poder realizar la aritmetica correspondiente

In [11]:
data_transform_obj_float=['Size', 'Price_open', 'S/L', 'T/P', 'Price_close', 'Swap', 'Profit']
for i in data_transform_obj_float:
    dft0[i]=dft0[i].astype('float64')

In [13]:
#transform_data_num
dft0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335 entries, 0 to 334
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Time_open     335 non-null    object 
 1   Type          331 non-null    object 
 2   Size          335 non-null    float64
 3   Symbol        335 non-null    object 
 4   Price_open    335 non-null    float64
 5   S/L           335 non-null    float64
 6   T/P           335 non-null    float64
 7   Time_close    335 non-null    object 
 8   Price_close   335 non-null    float64
 9   Swap          335 non-null    float64
 10  Profit        335 non-null    float64
 11  Comment       335 non-null    object 
 12  Symbol_only   335 non-null    object 
 13  temporality0  334 non-null    object 
 14  temporality1  88 non-null     object 
 15  temporality2  22 non-null     object 
dtypes: float64(7), object(9)
memory usage: 42.0+ KB


Ahora que realizamos el cambio de tipo de variable, he logrado notar que los datos del tiempo se encuentran de tipo object y la intención aquí es dejarlo de tipo data_time para mayor comodidad de operación

In [14]:
data_transform_time=['Time_open', 'Time_close']
for j in data_transform_time:
    dft0[j]=pd.to_datetime(dft0[j])

In [15]:
dft0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335 entries, 0 to 334
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Time_open     335 non-null    datetime64[ns]
 1   Type          331 non-null    object        
 2   Size          335 non-null    float64       
 3   Symbol        335 non-null    object        
 4   Price_open    335 non-null    float64       
 5   S/L           335 non-null    float64       
 6   T/P           335 non-null    float64       
 7   Time_close    335 non-null    datetime64[ns]
 8   Price_close   335 non-null    float64       
 9   Swap          335 non-null    float64       
 10  Profit        335 non-null    float64       
 11  Comment       335 non-null    object        
 12  Symbol_only   335 non-null    object        
 13  temporality0  334 non-null    object        
 14  temporality1  88 non-null     object        
 15  temporality2  22 non-null     object    

La conversión a un tipo de datos manejable desde pandas nos evidencio impureza en los datos a la hora de la digitación. Lo que conllevo a realizar ciertas correcciones desde la base de datos de donde se encuentra el rep

In [20]:
dft0['temporality0'].unique()

array(['M15', 'M5', 'M30', 'H1', 'm1', 'k1', 'gas', '', 'H4', None, 'n1',
       'u1', '20', 'z1', 'h2', 'm2'], dtype=object)

In [57]:
arr_temp0=['m1','k1','n1','u1','z1','h2','m2']

In [60]:
column_temp0=dft0[dft0['temporality0'].isin(arr_temp0)][['temporality0']]

In [67]:
column_temp1=dft0[dft0['temporality0'].isin(arr_temp0)][['temporality1']]
#column_temp1

In [78]:
column_aux=column_temp1.copy()
#column_aux

In [82]:
dft0_aux=dft0[dft0['temporality0'].isin(arr_temp0)].copy()
dft0_aux['temporality0']=column_aux
dft0_aux

,Time_open,Type,Size,Symbol,Price_open,S/L,T/P,Time_close,Price_close,Swap,Profit,Comment,Symbol_only,temporality0,temporality1,temporality2
8,2021-04-01 14:01:22,sell,0.01,us500_m1_H1,4000.5000,4004.5000,3992.7500,2021-04-01 14:48:26,4004.5000,0.00,-2.00,Senti control,us500,H1,H1,None
25,2021-04-07 08:41:28,buy,0.01,nat.gas_k1_H1,2.5020,2.4480,2.5470,2021-04-07 09:27:49,2.5470,0.00,14.65,Realajado y Cabeza fria,nat.gas,H1,H1,None
28,2021-04-07 10:02:12,buy,0.01,us30_m1_H1,33347.0000,33284.0000,33427.0000,2021-04-07 10:27:24,33284.0000,0.00,-3.15,Cabeza fría,us30,H1,H1,None
29,2021-04-07 10:04:39,buy,0.01,us500_m1_H1,4071.7500,4060.2500,4082.5000,2021-04-07 10:40:07,4060.2500,0.00,-5.75,Cabeza fria,us500,H1,H1,None
30,2021-04-07 10:06:15,buy,0.01,us100_m1_H1,13628.5000,13507.2500,13761.0000,2021-04-07 18:33:45,13634.2500,-0.12,1.15,Cabeza fria,us100,H1,H1,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,2022-04-25 08:43:28,buy,0.01,us30_m2_H1,33521.0000,33269.0000,33782.0000,2022-04-25 10:47:40,33269.0000,0.00,-12.60,Cabeza fría,us30,H1,H1,None
328,2022-04-25 12:16:52,buy,0.01,gbp_m2_H1,1.2732,1.2679,1.2789,2022-04-26 07:16:55,1.2679,-0.03,-3.31,Cabeza fría,gbp,H1,H1,None
329,2022-04-25 12:17:25,buy,0.01,ger40_m2_H1,14020.0000,13897.5000,14147.0000,2022-04-26 01:00:03,14147.0000,-0.05,34.07,Cabeza fría,ger40,H1,H1,None
330,2022-04-25 12:19:32,buy,0.01,uk100_m2_H1,7382.0000,7322.0000,7442.0000,2022-04-26 01:00:03,7442.0000,-0.03,7.65,Cabeza fría,uk100,H1,H1,None


In [84]:
dft0_aux2=dft0.copy()
dft0_aux2

,Time_open,Type,Size,Symbol,Price_open,S/L,T/P,Time_close,Price_close,Swap,Profit,Comment,Symbol_only,temporality0,temporality1,temporality2
0,2021-03-31 12:00:14,sell,0.01,gbpusd_M15,1.37986,1.38092,1.37825,2021-03-31 12:11:11,1.38092,0.00,-1.06,Me senti relajado,gbpusd,M15,None,None
1,2021-03-31 12:31:33,buy,0.03,usdjpy_M15,110.65700,110.53000,110.71000,2021-03-31 13:10:36,110.71000,0.00,1.44,senti control,usdjpy,M15,None,None
2,2021-03-31 12:33:46,sell,0.03,gpbusd_M5,1.37972,1.38072,1.37864,2021-03-31 13:24:21,1.37864,0.00,3.24,sentí control,gpbusd,M5,None,None
3,2021-03-31 16:11:22,sell,0.03,usdjpy_M15,110.66300,110.82200,110.56000,2021-03-31 18:08:40,110.82200,0.00,-4.30,Senti relajado,usdjpy,M15,None,None
4,2021-03-31 19:56:49,buy,0.03,usdchf_M30,0.94478,0.94357,0.94533,2021-03-31 11:23:01,0.94533,0.00,1.75,Senti control,usdchf,M30,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,2022-04-25 12:19:32,buy,0.01,uk100_m2_H1,7382.00000,7322.00000,7442.00000,2022-04-26 01:00:03,7442.00000,-0.03,7.65,Cabeza fría,uk100,m2,H1,None
331,2022-04-28 17:44:31,sell,0.01,usdjpy_H1,130.82500,131.34200,130.23700,2022-04-29 01:23:21,130.23700,0.00,4.51,Cabeza fria,usdjpy,H1,None,None
332,2022-05-05 10:02:04,sell,0.01,wticash_H1,107.24000,109.46000,105.02000,2022-05-05 16:28:07,109.46000,-0.55,-22.20,Cabeza fría,wticash,H1,None,None
333,2022-05-09 14:01:15,buy,0.01,ger40_m2_H1,13475.00000,13336.00000,13622.00000,2022-05-10 03:38:38,13622.00000,-0.05,38.83,Cabeza fría,ger40,m2,H1,None


In [89]:
dft0_aux2.loc[dft0_aux2['temporality0'].isin(arr_temp0),column_aux]

ValueError: Cannot index with multidimensional key